# Scraping Google Jobs

Google jobs has their filters in the URL, so we simply need Pylenium to visit a URL to start scraping.

> This is actually pretty powerful now, so it's something I'd recommend people use!

Unlike other job sites, like Indeed, Google's results are _almost_ always relevant when it comes to jobs

In [1]:
from pylenium.driver import Pylenium, PyleniumConfig, Keys

py = Pylenium(PyleniumConfig())
py.visit("https://www.google.com/search?q=QA+Engineer+jobs+utah")
py.getx("//a[contains(., 'more jobs')]").click()

In [2]:
jobs = []
share_link = None
results = py.find("[aria-label='Jobs list'] li [role='treeitem']")

for result in results:
    # 1. Collect its short content
    content = result.text()
    jobs.append(content)

    if share_link is None:
        # 2. Click on the Share button within the Details Page
        page = py.get("div#gws-plugins-horizon-jobs__job_details_page")
        page.get("div[aria-label='Share'][role='button']").click(force=True)

        # 3. Find the shareable link
        share = py.find("[aria-label='Share'][role='dialog'] input").last()
        py.wait().until(lambda _: share.get_attribute("value").startswith("https://g.co"))
        share_link = share.get_attribute("value")

        # 4. Close the dialog
        share.type(Keys.ESCAPE)

In [3]:
print(share_link)
print("Jobs found:", len(jobs), end="\n---\n")
for job in jobs[:2]:
    print(job, end="\n\n")

https://g.co/kgs/zLAeDnY
Jobs found: 20
---
QA Automation Engineer
Dyno Nobel
Salt Lake City, UT
via LinkedIn
21 hours ago
Full-time
Health insurance
Dental insurance
Paid time off

Software Quality Assurance Engineer, Document Systems
Tesla
Draper, UT
via LinkedIn
20 days ago
Full-time
Health insurance
Dental insurance
Paid time off

